## Pemrosesan Suara dan Bahasa Alami
### Tugas Kelompok Praktikum Pertemuan 6
### Kelompok 1


|Nama|NIM|
|-|-|
|Nike Putra Jaya Hulu | G64130097|
|Nurul Andayani | G64164001|
|Rya Meyvriska| G64164008|
|Faruq Sandi|G64164013|
|Ismail Adima|G64164053|

In [16]:
# 0. import library
import numpy as np
import copy
from scipy.io import wavfile
from matplotlib import pyplot as plt
import pandas as pd
from sklearn import datasets
from neupy import algorithms, environment

In [17]:
# 1. membuat fungsi mfcc
def Hamming_Window(n):
    w = []
    for i in range(0,n):
        temp = 0.54 - 0.46 * np.cos((2*np.pi*i)/(n-1))
        w.append(temp)
    return w

# mengambil mfcc dari suara 0-1.5 second
def getMFCC(fileDirectory):
    #get sample rate,signal,pre emphasis, dan emphasized
    sample_rateku, signalku = wavfile.read(fileDirectory)
    signalku = signalku[0:int(1.5 * sample_rateku)]
    timesku = np.arange(len(signalku))/float(sample_rateku)

    pre_emphasisku = 0.97
    emphasized_signalku = np.append(signalku[0], signalku[1:]- pre_emphasisku * signalku[:-1])
    
    # Ukuran frame adalah 25 milidetik,
    # dan overlap 15 milidetik (dalam ukuran waktu/detik)
    frame_sizeku = 0.025
    frame_strideku = 0.01

    # Konversi dari detik ke sample
    frame_lengthku = int(round(frame_sizeku * sample_rateku))
    frame_stepku = int(round(frame_strideku * sample_rateku))

    # Ukuran sinyal dalam Sample
    signal_lengthku = len(emphasized_signalku)

    # Pastikan setidaknya ada 1 frame
    num_framesku = int(np.ceil(float(np.abs(signal_lengthku - frame_lengthku)) / frame_stepku))

    pad_signal_lengthku = num_framesku * frame_stepku + frame_lengthku
    zku = np.zeros((pad_signal_lengthku - signal_lengthku))
    pad_signalku = np.append(emphasized_signalku, zku)

    indicesku = np.tile(np.arange(0, frame_lengthku), (num_framesku, 1))
    + np.tile(np.arange(0, num_framesku * frame_stepku, frame_stepku)
    , (frame_lengthku, 1)).T
    framesku = pad_signalku[indicesku.astype(np.int32, copy=False)]

    # windowing
    framesHWku = copy.copy(framesku)
    framesHWku *= np.hamming(frame_lengthku)

    # Magnitude of the FFT
    NFFTku = 512
    mag_framesku = np.absolute(np.fft.rfft(framesku, NFFTku))

    # Power Spectrum
    pow_framesku = ((1.0 / NFFTku) * ((mag_framesku) ** 2))

    # Misalkan yang diambil adalah:
    nfiltku = 40

    low_freq_melku = 0
    high_freq_melku = (2595 * np.log10(1 + (sample_rateku / 2) / 700)) # Convert Hz to Mel
    mel_pointsku = np.linspace(low_freq_melku, high_freq_melku, nfiltku + 2)
    hz_pointsku = (700 * (10**(mel_pointsku / 2595) - 1)) # Convert Mel to Hz
    bin = np.floor((NFFTku + 1) * hz_pointsku / sample_rateku)


    fbankku = np.zeros((nfiltku, int(np.floor(NFFTku / 2 + 1))))
    for m in range(1, nfiltku + 1):
        f_m_minus = int(bin[m - 1]) # left
        f_m = int(bin[m]) # center
        f_m_plus = int(bin[m + 1]) # right
        for k in range(f_m_minus, f_m):
            fbankku[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
        for k in range(f_m, f_m_plus):
            fbankku[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])
    filter_banksku = np.dot(pow_framesku, fbankku.T)
    filter_banksku = np.where(filter_banksku == 0, np.finfo(float).eps, filter_banksku)
    filter_banksku = 20 * np.log10(filter_banksku) # dB

    num_cepsku = 13

    from scipy.fftpack import dct
    # Keep 2-14
    mfccku = dct(filter_banksku, type=2, axis=1, norm='ortho')[:, 1 : (num_cepsku + 1)]
    mfccku -= (np.mean(mfccku, axis=0) + 1e-8)
 
    return mfccku

In [18]:
# 2. Plot semua data
#anggota = ["faruq","ismail","nurul","putra","rya"]
anggota = ["faruq","putra","rya","nurul","ismail"]
anggota_code = [1,2,3,4,5]
mfccAnggota = []

def ambilMfcc(x):
    mfcc_row = []
    for k in range(len(x[0])):
        sumCol = 0
        for l in range(len(x)):
            #print(l,k)
            sumCol += x[l][k]
        mfcc_row.append((sumCol/(len(mfcc[k])+1)))
    return mfcc_row

for i in range(len(anggota)):
    nama = anggota[i]
    print("Memproses data:",nama)
    for j in range(0,30):
        mfcc = getMFCC("./Dataset/" + nama + "/"+nama + "_" + str(j+1) + ".wav")
        mfccAnggota.append(ambilMfcc(mfcc))

Memproses data: faruq
Memproses data: putra
Memproses data: rya
Memproses data: nurul
Memproses data: ismail


In [19]:
print(len(mfccAnggota[0]))

13


In [20]:
# 3. pengambilan dataset
environment.reproducible()
dataset = mfccAnggota
labelset = []


for i in range(len(anggota)):
    for j in range(30):
        labelset.append(j)


In [21]:
#traning data
#data_train = np.vstack([dataset[0:18],dataset[24:43],dataset[48:67],dataset[72:91],dataset[96:115]])
#target_train = np.hstack([labelset[0:18],labelset[24:43],labelset[48:67],labelset[72:91],labelset[96:115]])
#data_test = np.vstack([dataset[19:23],dataset[44:47],dataset[68:71],dataset[72:91],dataset[116:119]])
#target_test = np.hstack([labelset[19:23],labelset[44:47],labelset[68:71],labelset[72:91],labelset[116:119]])
data_test=[]
data_train=[]
label_test=[]
label_train=[]

for i in range (len(anggota)):
    for j in range(30):
        if(j<6):
            data_test.append(dataset[j])
            label_test.append(labelset[i])
        else:
            data_train.append(dataset[j])
            label_train.append(labelset[i])


In [22]:
print(len(data_test))
print(len(data_train))
print(label_train)
print(label_test)

30
120
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4]


In [23]:
lvqnet = algorithms.LVQ3(n_inputs=13,n_classes=5,verbose=True,show_epoch=20,step=0.00000001,n_updates_to_stepdrop=15*100)


Main information

[ALGORITHM] LVQ3

[OPTION] verbose = True
[OPTION] epoch_end_signal = None
[OPTION] show_epoch = 20
[OPTION] shuffle_data = False
[OPTION] train_end_signal = None
[OPTION] minstep = 1e-05
[OPTION] n_classes = 5
[OPTION] n_inputs = 13
[OPTION] n_subclasses = None
[OPTION] n_updates_to_stepdrop = 1500
[OPTION] prototypes_per_class = None
[OPTION] weight = None
[OPTION] epsilon = 0.1
[OPTION] slowdown_rate = 0.4
[OPTION] step = 1e-08



In [24]:
lvqnet.train(data_train,label_train,epochs=300)


Start training

[TRAINING DATA] shapes: (120, 13)
[TRAINING] Total epochs: 300

---------------------------------------------------------
|    Epoch    |  Train err  |  Valid err  |    Time     |
---------------------------------------------------------
|           1 |         0.8 |           - |       53 ms |
|          20 |         0.8 |           - |       24 ms |
|          40 |         0.8 |           - |       20 ms |
|          60 |         0.8 |           - |       13 ms |
|          80 |         0.8 |           - |       12 ms |
|         100 |         0.8 |           - |       16 ms |
|         120 |     0.80833 |           - |       20 ms |
|         140 |     0.80833 |           - |       15 ms |
|         160 |     0.79167 |           - |       15 ms |
|         180 |     0.79167 |           - |       18 ms |
|         200 |     0.79167 |           - |       18 ms |
|         220 |         0.8 |           - |       21 ms |
|         240 |     0.78333 |           - |      

In [25]:
prediksi = lvqnet.predict(data_test)

In [26]:
print(prediksi)

[4 4 3 2 1 1 4 4 3 2 1 1 4 4 3 2 1 1 4 4 3 2 1 1 4 4 3 2 1 1]


In [27]:
y_pred = pd.Series(prediksi, name='Prediksi')
y_actu = pd.Series(label_test, name='Aktual')
df_confusion = pd.crosstab(y_actu, y_pred)

In [28]:
df_confusion

Prediksi,1,2,3,4
Aktual,,,,
0,2,1,1,2
1,2,1,1,2
2,2,1,1,2
3,2,1,1,2
4,2,1,1,2


In [29]:
from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(label_test, prediksi)
cm.print_stats()

Confusion Matrix:

Predicted  0   1  2  3   4  __all__
Actual                             
0          0   2  1  1   2        6
1          0   2  1  1   2        6
2          0   2  1  1   2        6
3          0   2  1  1   2        6
4          0   2  1  1   2        6
__all__    0  10  5  5  10       30


Overall Statistics:

Accuracy: 0.2
95% CI: (0.077135512001065609, 0.3856665109966072)
No Information Rate: ToDo
P-Value [Acc > NIR]: 0.964545828184
Kappa: 0.0
Mcnemar's Test P-Value: ToDo


Class Statistics:

Classes                                  0         1         2         3  \
Population                              30        30        30        30   
P: Condition positive                    6         6         6         6   
N: Condition negative                   24        24        24        24   
Test outcome positive                    0        10         5         5   
Test outcome negative                   30        20        25        25   
TP: True Positive         